# DT Assignment1

# Data Loading

In [21]:
import pandas as pd 
import numpy as np

In [22]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수를 구하는 함수 만들기

<img src = "https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fk.kakaocdn.net%2Fdn%2Fb8ZU7n%2Fbtqv0WgKx36%2FpwXSdPYV8fUCmTiR16UCD0%2Fimg.png">

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [82]:
def get_gini(df, label):
    ratio = df[label].value_counts(normalize=True) #normalize로 false, true 상대비율 구하기
    gini = 1- ((ratio**2).sum())
    
    return gini

In [83]:
get_gini(pd_data,'class_buys_computer') #class_buys_computer일때 지니 지수

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [84]:
pd_data["age"].value_counts().index

Index(['youth', 'senior', 'middle_aged'], dtype='object')

In [85]:
#결과 형식을 위해 리스트에 리스트를 
def get_binary_split(df, attribute):
    
    def powerset(s):
        x = len(s)
        ret = [[s[j] for j in range(x) if (i & (1<< j))] for i in range(1, 1<<x)]
        ret.pop()
        return ret
    
    table = df[attribute].value_counts() #클래스로 어떤게 있는지 구하기
    class_list = table.index
    result = powerset(class_list) #조합을 구해서 result에 넣는다.
    return result

In [86]:
get_binary_split(pd_data,'age')

[['youth'],
 ['senior'],
 ['youth', 'senior'],
 ['middle_aged'],
 ['youth', 'middle_aged'],
 ['senior', 'middle_aged']]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [87]:
def get_attribute_gini_index(df, attribute, label):
    
    # n= 총 몇 열로 되어있는지
    n = len(df)
    result ={}
    
    for i in get_binary_split(df, attribute):
        
        # split 해서 나온 결과를 가지고 있는 애들을 true라고 하고,, true인 애들을 뽑아냄.
        df1 = df[df[attribute].apply(lambda x: x in i)]      # 원하는 애들이 있는거
        df2 = df[df[attribute].apply(lambda x: x not in i)]      # 원하는 애들이 없는거
      
      
        lenth = len(df1)
        v = (lenth/n)*get_gini(df1,label)+(1-(lenth/n))*get_gini(df2,label)
        
        result[tuple(i)] = v  
    
    return result

In [88]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{('youth',): 0.3936507936507936,
 ('senior',): 0.4571428571428571,
 ('youth', 'senior'): 0.35714285714285715,
 ('middle_aged',): 0.35714285714285715,
 ('youth', 'middle_aged'): 0.4571428571428571,
 ('senior', 'middle_aged'): 0.39365079365079364}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

In [89]:
min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").items())

(('middle_aged',), 0.35714285714285715)

In [90]:
min(get_attribute_gini_index(pd_data, "income", "class_buys_computer").items())

(('high',), 0.4428571428571429)

In [91]:
min(get_attribute_gini_index(pd_data, "student", "class_buys_computer").items())

(('no',), 0.3673469387755103)

In [92]:
min(get_attribute_gini_index(pd_data, "credit_rating", "class_buys_computer").items())

(('excellent',), 0.42857142857142855)

# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [93]:
##문제1 답안
get_binary_split(pd_data, 'income')

[['medium'],
 ['low'],
 ['medium', 'low'],
 ['high'],
 ['medium', 'high'],
 ['low', 'high']]

In [94]:
##문제2 답안
#가장 중요한 변수 : age

my_dict = get_attribute_gini_index(pd_data, "age", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

Min - ('youth', 'senior') : 0.35714285714285715


In [95]:
##문제3 답안

##
my_dict = get_attribute_gini_index(df3, "age", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print(key_min, ":", my_dict[key_min])

my_dict = get_attribute_gini_index(df3, "income", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print(key_min, ":",my_dict[key_min])

my_dict = get_attribute_gini_index(df3, "student", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print( key_min, ":",my_dict[key_min])

my_dict = get_attribute_gini_index(df3, "credit_rating", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print( key_min, ":",my_dict[key_min])

('youth',) : 0.48
('medium', 'low') : 0.375
('no',) : 0.31999999999999984
('fair',) : 0.4166666666666667
